In [1]:
import json
from src.test import test

DEVICE = 'cuda:0'

# prepare validation and test sets

from src.processing import prepare_somos_data

prepare_somos_data(
    listeners=10,
    systems=10,
    texts=50,
    num_iterations=1000,
    somos_dir='data/SOMOS',
    output_dir='data/somos_prepared',
    random_seed=1234
) 

2024-01-06 19:56:58.694203 :: VALIDATION SET PREPARED :: 55 SAMPLES
VAL min wasserstein distance: 0.0535206090949691
2024-01-06 19:57:31.538124 :: TEST SET PREPARED :: 52 SAMPLES
TEST min wasserstein distance: 0.00944398217953811
validation locales
us: 55
test locales
us: 28
gb: 24
2024-01-06 19:57:31.539653 :: SAVED TO data/somos_prepared


In [2]:
# define MOSNet model baseline

from src.mosnet.models import MOSNet, ProsAlignMOSNet
from src.mosnet.train import train_mosnet

MOSNET_STFT_PARAMS = {
    'n_fft': 512,
    'hop_length': 256
}

BASELINE_MOSNET_PARAMS = {
    'cnn_channels': [16, 32, 64, 128],
    'cnn_kernel_size': 3,
    'blstm_hidden_size': 128,
    'fc_hidden_size': 128,
    'fc_dropout': 0.3
}

TEST_VAL_DIR = 'data/somos_prepared'
model = MOSNet(
    stft_params=MOSNET_STFT_PARAMS,
    model_params=BASELINE_MOSNET_PARAMS
).to(DEVICE)

ImportError: cannot import name 'SOMOSTestValDataset' from 'src.datasets' (/home/overlord/Documents/content_aware_mos_prediction/src/datasets.py)

In [3]:
# train baseline MOSNet model

model = train_mosnet(
    model=model, 
    model_name='mosnet',
    train_params={
        'epochs': 50,
        'batch_size': 64,
        'num_workers': 5,
        'learning_rate': 1e-4
    },
    #validation_params=TEST_VAL_PARAMS,
    
    device=DEVICE,
    run_dir='runs/prod/mosnet'
)

2023-12-10 16:29:04.425595 :: START RUN
2023-12-10 16:29:50.518140 :: EPOCH 1 :: MOSNET_TRAIN :: METRICS :: 
UTTERANCE :: { lcc: 0.0272, srcc: 0.0416, mse: 0.3171, ktau: 0.0279 } :: SYSTEM :: { lcc: 0.0615, srcc: 0.0574, mse: 0.3305, ktau: 0.0417 }
2023-12-10 16:30:36.983739 :: EPOCH 2 :: MOSNET_TRAIN :: METRICS :: 
UTTERANCE :: { lcc: 0.1313, srcc: 0.1187, mse: 0.3105, ktau: 0.0799 } :: SYSTEM :: { lcc: 0.1593, srcc: 0.1323, mse: 0.3088, ktau: 0.0897 }
2023-12-10 16:31:23.031364 :: EPOCH 3 :: MOSNET_TRAIN :: METRICS :: 
UTTERANCE :: { lcc: 0.2608, srcc: 0.2555, mse: 0.3009, ktau: 0.1727 } :: SYSTEM :: { lcc: 0.2903, srcc: 0.2221, mse: 0.2907, ktau: 0.1584 }
2023-12-10 16:32:09.185039 :: EPOCH 4 :: MOSNET_TRAIN :: METRICS :: 
UTTERANCE :: { lcc: 0.2992, srcc: 0.2996, mse: 0.2944, ktau: 0.2029 } :: SYSTEM :: { lcc: 0.3118, srcc: 0.3384, mse: 0.3031, ktau: 0.2330 }
2023-12-10 16:32:55.171757 :: EPOCH 5 :: MOSNET_TRAIN :: METRICS :: 
UTTERANCE :: { lcc: 0.3365, srcc: 0.3351, mse: 0.2883, 

In [4]:
# test baseline MOSNet model

## TODO: make sure test infers model device
'''
test(
    model=model,
    model_name='mosnet',
    run_dir='/runs',
    run_key='prod',
    test_params=TEST_VAL_PARAMS
)
'''

"\ntest(\n    model=model,\n    model_name='mosnet',\n    run_dir='/runs',\n    run_key='prod',\n    test_params=TEST_VAL_PARAMS\n)\n"

In [5]:
# define MOSNet model with prosody alignment

'''
model = ProsAlignMOSNet(
    stft_params=MOSNET_STFT_PARAMS,
    model_params={
        'pros_align_embedding_size': 64,
        'pros_align_ff_dim': 128,
        'pros_align_ff_dropout': 0.1,
        'pros_align_bigru_dim': 128,
        **BASELINE_MOSNET_PARAMS
    }
)
'''

"\nmodel = ProsAlignMOSNet(\n    stft_params=MOSNET_STFT_PARAMS,\n    model_params={\n        'pros_align_embedding_size': 64,\n        'pros_align_ff_dim': 128,\n        'pros_align_ff_dropout': 0.1,\n        'pros_align_bigru_dim': 128,\n        **BASELINE_MOSNET_PARAMS\n    }\n)\n"